In [ ]:
%%bash
mkdir data
cd data
    
#wget http://biodatageeks.ii.pw.edu.pl/sequila/data/WES/NA12878.proper.wes.chr1.md.bam
wget http://biodatageeks.ii.pw.edu.pl/sequila/data/WES/NA12878.proper.wes.chr1.md.bam
wget http://biodatageeks.ii.pw.edu.pl/sequila/data/WES/Homo_sapiens_assembly18.chr1.fasta
wget http://biodatageeks.ii.pw.edu.pl/sequila/data/WES/Homo_sapiens_assembly18.chr1.fasta.fai


In [ ]:
pip install pandas
pip install matplotlib

In [3]:
import os
base_path = f"{os.getcwd()}/data"
bam_path = f'{base_path}/NA12878.proper.wes.chr1.md.bam'
ref_path=f'{base_path}/Homo_sapiens_assembly18.chr1.fasta'
sample_id= 'NA12878'
table_name = 'reads'
app_name = "sequila"

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession \
.builder \
.appName(f'{app_name}') \
.getOrCreate()

In [5]:
from pysequila import SequilaSession
ss = SequilaSession(spark)
ss.sql(f'''DROP TABLE IF EXISTS {table_name}''')
ss.sql(f'''CREATE TABLE IF NOT EXISTS {table_name} \
         USING org.biodatageeks.sequila.datasources.BAM.BAMDataSource \
         OPTIONS(path "{bam_path}")''')

DataFrame[]

     |████████████████████████████████| 9.5 MB 1.6 MB/s eta 0:00:01     |▉                               | 235 kB 451 kB/s eta 0:00:21
     |████████████████████████████████| 14.9 MB 1.2 MB/s eta 0:00:01
     |████████████████████████████████| 510 kB 3.3 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


     |████████████████████████████████| 11.6 MB 6.2 MB/s eta 0:00:01    |██████████████████████          | 8.0 MB 6.2 MB/s eta 0:00:01
     |████████████████████████████████| 67 kB 2.1 MB/s eta 0:00:01
     |████████████████████████████████| 88 kB 2.6 MB/s eta 0:00:01
     |████████████████████████████████| 2.2 MB 3.9 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [23]:
import pandas as pd
pd.options.display.max_columns = None
ss.sql(f"SELECT * FROM {table_name} LIMIT 5").toPandas()

,sample_id,qname,flag,contig,pos,pos_start,pos_end,mapq,cigar,rnext,pnext,tlen,seq,qual,tag_AM,tag_AS,tag_BC,tag_BQ,tag_BZ,tag_CB,tag_CC,tag_CG,tag_CM,tag_CO,tag_CP,tag_CQ,tag_CR,tag_CS,tag_CT,tag_CY,tag_E2,tag_FI,tag_FS,tag_FZ,tag_H0,tag_H1,tag_H2,tag_HI,tag_IH,tag_LB,tag_MC,tag_MD,tag_MI,tag_MQ,tag_NH,tag_NM,tag_OA,tag_OC,tag_OP,tag_OQ,tag_OX,tag_PG,tag_PQ,tag_PT,tag_PU,tag_Q2,tag_QT,tag_QX,tag_R2,tag_RG,tag_RX,tag_SA,tag_SM,tag_TC,tag_U2,tag_UQ
0,NA12878,61DC0AAXX100127:8:58:8295:16397,99,MT,7,7,107,99,101M,MT,27,101,AGGTCTATCACCCTATTAACCACTCACGGGAGCTCTCCATGCATTT...,ADEAEDBCFCEFFFCDEDFGHEGGHEGBIIFIHGHHIHFFIIFFGG...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,CAAGAGCGACAAAGCGGCCAACAGACAATACTAGACAACGAACAGG...,None,None,None,None,None,None,None,None,None,None,66G34,None,99.0,None,1,None,None,None,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...,None,61DC0.8,None,None,None,None,None,None,None,61DC0.8,None,None,None,None,None,36
1,NA12878,61DC0AAXX100127:8:35:1455:1743,1113,MT,9,9,109,90,101M,MT,9,101,GTCTGTCACCCTTGTAGCCGCTCACGGGAGCTCTCCATGCATTTGG...,##########################G1/7=@AEHB5=F=DFC@FE...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,CAAACAATGTAGAAATAAACACACAAACCTAAAAATGGTTGAACTA...,None,None,None,None,None,None,None,None,None,None,4A7A0T2A2A44G36,None,NaN,None,6,None,None,None,##########################B1+8;>>BB>1<B9BB@>BB...,None,61DC0.8,None,None,None,None,None,None,None,61DC0.8,None,None,None,None,None,34
2,NA12878,61DC0AAXX100127:8:1:5111:5929,1177,MT,10,10,110,99,101M,MT,10,101,TCTATCCCCCTATTAACCACTCACGGGAGCTCTCCATGCATTTGGT...,"B?=<=:""$.,/;;@?>CI@CAIEDIIGGGHDEFBIEGCIFHHGIHE...",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,GTCTAAAAAACTAATTAACAAATATTTGTTATAAATGTTTAGGTTG...,None,None,None,None,None,None,None,None,None,None,6A56G37,None,NaN,None,2,None,None,None,B<?79:&*0+.8:@=:@C=@=CBCDCBBBB@AD@CDCADCCCCCCC...,None,61DC0.8,None,None,None,None,None,None,None,61DC0.8,None,None,None,None,None,38
3,NA12878,61DC0AAXX100127:8:47:17866:4686,153,MT,20,20,120,99,101M,MT,20,101,TATTATCCACTCACGGGAGCTCTCCATGCATTTGGTATTTTCGTCT...,###########ICA@GGG=DFDFHC=@GH>HG<IAFBHHFDCHCFC...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,ATGATAAACTAACACTTGTTATATATGTATGGGTTGTGGGCTTCTG...,None,None,None,None,None,None,None,None,None,None,5A47G47,None,NaN,None,2,None,None,None,###########B@B@BBB=AB@BB>;?BB:CB<CAC@CCBACCABA...,None,61DC0.8,None,None,None,None,None,None,None,61DC0.8,None,None,None,None,None,37
4,NA12878,61CC3AAXX100125:5:79:19223:6261,99,MT,25,25,100,99,76M,MT,30,76,ACCACTCACGGGAGCTCTCCATGCATTTGGTATTTTCGTCTGGGGG...,ABCADDFCE@GGEHGFHFHHEEIHEEFFII>DEFFFIBDHGIIIII...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,CAACAGACATTTCTAGAGAATGAACGGGTTGCAGGGAAGAGTTTTT...,None,None,None,None,None,None,None,None,None,None,48G27,None,99.0,None,1,None,None,None,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCC@CCCCCCCCCCCCCCC...,None,61CC3.5,None,None,None,None,None,None,None,61CC3.5,None,None,None,None,None,29


In [ ]:
import matplotlib.pyplot as plt
df = ss.sql(f"SELECT * FROM  pileup('{table_name}', '{sample_id}', '{ref_path}', true) WHERE contig='1' ").toPandas()
df['baf'] = df.countNonRef/df.coverage
df2 = df[(((df.baf > 0.35) & (df.baf< 0.65)) | (df.baf > 0.9)) & (df.coverage>20) & (df.coverage<300)]
plt.rcParams['figure.figsize'] = [15,3]; plt.scatter(df2.pos_start, df2.baf, s=30,alpha=0.1);
plt.xlabel('Chromsome 1');plt.ylabel('B-allele frequency'); plt.ylim(0,1); plt.show()
